In [ ]:
#default_exp web.app

# Web App

> Factory function that creates and configures the flask app.

In [ ]:
#export
import os
from flask import Flask
from pathlib import Path

In [ ]:
#export
def create_app(test_config=None):
    "Create and configure an instance of the Flask application."
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        # a default secret that should be overridden by instance config
        SECRET_KEY="dev",
        # store the database in the instance folder
        DATABASE=os.path.join(app.instance_path, "web_journal.sqlite"),
    )

    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile("config.py", silent=True)
    else:
        # load the test config if passed in
        app.config.update(test_config)

    # ensure the instance folder exists
    os.makedirs(app.instance_path, exist_ok=True)
    
    @app.route("/hello")
    def hello(): return "Hello, World!"

    from web_journal.web import db, auth, blog
    # register the database commands
    db.init_app(app)
    # apply the blueprints to the app
    app.register_blueprint(auth.bp)
    app.register_blueprint(blog.bp)

    # make url_for('index') == url_for('blog.index')
    # in another app, you might define a separate main index here with
    # app.route, while giving the blog blueprint a url_prefix, but for
    # the tutorial the blog will be the main index
    app.add_url_rule("/", endpoint="index")

    return app

In [ ]:
assert not create_app().testing
assert create_app({"TESTING": True}).testing

In [ ]:
#hide
from flask import Flask
from contextlib import contextmanager
from pathlib import Path
import tempfile, shutil, os, web_journal.web

def cd(p):
    print('Changing working directory:') # TODO: maybe we want to suppress stdout
    get_ipython().run_line_magic('cd', str(p))

@contextmanager
def test_resources():
    db_fd, db_path = tempfile.mkstemp()
    try:
        cwd = get_ipython().run_line_magic('pwd', '')
        cd(Path(web_journal.web.__file__).parent)
        app = create_app({"TESTING": True, "DATABASE": db_path})
        client = app.test_client()
        with app.app_context():
            yield app, client
    finally:
        cd(cwd)
        os.close(db_fd)
        os.unlink(db_path)

In [ ]:
with test_resources() as (app, client):
    response = client.get("/hello")
    assert response.data == b"Hello, World!"

Changing working directory:
C:\Users\Butterp\github\pete88b\web_journal\web_journal\web
Changing working directory:
C:\Users\Butterp\github\pete88b\web_journal


In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_db.ipynb.
Converted 50b_web_auth.ipynb.
Converted 50c_web_blog.ipynb.
Converted index.ipynb.
